In [ ]:
"""
    First basic livetracking graph.
    Consist of motor, sensor and ultrasound values
"""

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import pandas as pd

plt.ion()

while True:
    data = pd.read_csv("output.csv")
    
    left_motor = data['left motor'].values
    left_motor_norm = left_motor / 1000
    right_motor = data['right motor'].values
    right_motor_norm = right_motor / 1000
    total_motor_norm = (left_motor+right_motor) / 1000
    
    left_sensor = data['left sensor'].values
    right_sensor = data['right sensor'].values
    total_sensor = left_sensor + right_sensor
    
    left_us_sensor = data['left ultrasound sensor'].values
    right_us_sensor = data['right ultrasound sensor'].values
    sensor = data['left sensor'].values + data['right sensor'].values

    plt.plot(left_sensor)
    plt.plot(right_sensor)
    plt.axis('tight')
    plt.legend(['left_motor_norm','right_motor_norm','left_sensor','right_sensor','left motor','right motor'])

    clear_output(wait=True)
    plt.pause(1)

while True:
    clear_output(wait=True)
    plt.pause(1)



In [ ]:
"""
    Livetracking of a heatmap of a normalized motor, sensor and ultrasound values.
"""

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import pandas as pd
while True:
    plt.figure(figsize=(10,10))
    data = pd.read_csv("output.csv")

    data['right motor'] = data['right motor'].values/1000
    data['left motor'] = data['left motor'].values/1000

    im = plt.imshow(data,cmap='hot',aspect='auto')
    plt.colorbar(im,orientation='horizontal')
    
    clear_output(wait=True)
    plt.pause(1)


while True:
    clear_output(wait=True)
    plt.pause(1)


In [ ]:
"""
    State transition between bins
"""

from DataManipulation import DataManipulation
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import math
import collections
import pandas as pd
import networkx as nx
from IPython.display import clear_output

plt.ion()

# Print the whole array rather than just a subset of the array
np.set_printoptions(threshold=np.nan,suppress=True)

# Necessary for livetracking
while True:
    grid = plt.figure(figsize=(10,10))
    #Add position for the plot with 211 being 2 row 1 column and position 1
    ls = grid.add_subplot(211)
    plt.title("Transition plot between Sensor and Motor states")
    plt.xlabel("Sensor")
    plt.ylabel("Motor")
    rs = grid.add_subplot(212)
    plt.title("Transition plot between Ultrasound and Motor states")
    plt.xlabel("Ultrasound")
    plt.ylabel("Motor")
    
    # Reading output.csv and normalizing the data
    data = DataManipulation("../server/output.csv")
    NUM_OF_BINS = data.get_NUM_OF_BINS()
    data.normalized_motor()
    motor_values,sensor_values,us_values = data.normalized_0_to_1()
    
    #Making a state transition between sensor and motor values
    sensor_and_motor = data.convert_values_to_bins(sensor_values,motor_values)
    transition_array = data.remove_continous_state()
    transition_with_frequency = data.transition_frequency(transition_array)
    G = nx.DiGraph()
    G.add_nodes_from(transition_with_frequency.keys())
    for i,j in transition_with_frequency.items():
        for k,l in j.items():
            G.add_weighted_edges_from([(i, k, l)])

    # This resulted in an array where for every 3 values of the array such as [A,B,C],
    # A is source state, B is destination state and C is frequency of transition from A to B
    most_visited_state = data.most_visited_state_transition(transition_with_frequency)
    
    # Separating the values in the array to bins
    xArray,yArray = data.digitize_total_values(most_visited_state, type='2d')

    """for i in nx.simple_cycles(G):
        i.append(i[0])
        tobe = np.array(i)
        xArray2,yArray2 = data.digitize_total_values(tobe, type='2d')
        for i in range(len(xArray2)-1):
            ls.annotate(
                        '',
                        xy=(xArray2[i+1],yArray2[i+1]),
                        xycoords='data',
                        xytext=(xArray2[i],yArray2[i]),
                        textcoords='data',
                        verticalalignment="center", 
                        horizontalalignment="center",
                        arrowprops=dict(facecolor='black', arrowstyle="-|>", 
                                        shrinkB=0.1, alpha=0.1)
                    )"""

        
    # Making the arrow transition between bins
    # Can use quiver but this one looks better
        
    for i in range(0,len(xArray)-1,3):
        test = ls.annotate(
            str(most_visited_state[i+2]), # text to be displayed
            xy=(xArray[i+1],yArray[i+1]), # to here
            xycoords='data',
            xytext=(xArray[i],yArray[i]), # arrow starts from here
            textcoords='data',
            horizontalalignment='left', # so we can actually see the frequency displayed 
            verticalalignment='bottom',
            arrowprops=dict(facecolor='black', arrowstyle="-|>", shrinkB=0.1, alpha=0.4)
        )
        
    # Making a state transition between ultrasound and motor values
    us_and_motor = data.convert_values_to_bins(us_values,motor_values)
    transition_array = data.remove_continous_state()
    transition_with_frequency = data.transition_frequency(transition_array)
    most_visited_state = data.most_visited_state_transition(transition_with_frequency)
    xArray,yArray,zArray = data.digitize_total_values(most_visited_state, type='3d')

    for i in range(0,len(xArray)-1,3):
        test = rs.annotate(
            str(most_visited_state[i+2]),
            xy=(xArray[i+1],yArray[i+1]), # to here
            xycoords='data',
            xytext=(xArray[i],yArray[i]), # arrow starts from here
            textcoords='data',
            horizontalalignment='left',
            verticalalignment='bottom',
            arrowprops=dict(facecolor='black', arrowstyle="-|>", shrinkB=0.1, alpha=0.3)
        )
    
    ls.set_xlim(-1,NUM_OF_BINS)
    ls.set_ylim(-1,NUM_OF_BINS)
    rs.set_xlim(-1,NUM_OF_BINS)
    rs.set_ylim(-1,NUM_OF_BINS)
    
    # Necessary for livetracking
    clear_output(wait=True)
    plt.pause(1)

# Necessary for livetracking
while True:
    clear_output(wait=True)
    plt.pause(1)
    

In [ ]:
"""
    6 side-by-side graph of all the sensor, motor and ultrasound values
"""

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import pandas as pd

plt.ion()

while True:
    grid = plt.figure(figsize=(20,10))
    ls = grid.add_subplot(321)
    plt.title("Left Light Sensor")
    rs = grid.add_subplot(322)
    plt.title("Right Light Sensor")
    luv = grid.add_subplot(323)
    plt.title("Left UltraViolet Sensor")
    ruv = grid.add_subplot(324)
    plt.title("Right UltraViolet Sensor")
    lm = grid.add_subplot(325)
    plt.title("Left Motor")
    rm = grid.add_subplot(326)
    plt.title("Right Motor")

    data = pd.read_csv("output.csv")
    time = np.arange(0, data.shape[0]/20, 0.05) # To get how many seconds has the program been running
    lmv = data['left motor'].values
    rmv = data['right motor'].values
    lsv = data['left sensor'].values
    rsv = data['right sensor'].values
    luvv = data['left ultrasound sensor'].values
    ruvv = data['right ultrasound sensor'].values
    
    ls.plot(time, lsv, label='left-light')
    ls.set_ylim(0, 0.5)
    
    rs.plot(time, rsv, label='right-light')
    rs.set_ylim(0, 0.5)

    luv.plot(time, luvv, label='left-uv')
    luv.set_ylim(0, 0.8)
    luv.autoscale(enable=True,axis='both')
    
    ruv.plot(time, ruvv, label='right-uv')
    ruv.set_ylim(0, 0.8)
    ruv.autoscale(enable=True,axis='both')
    
    lm.plot(time, lmv, label='left-motor')
    lm.set_ylim(-500, 500)
    lm.autoscale(enable=True,axis='both')

    rm.plot(time, rmv, label='right-motor')
    rm.set_ylim(-500, 500)
    rm.autoscale(enable=True,axis='both')
    
    clear_output(wait=True)
    plt.pause(1)
    
while True:
    clear_output(wait=True)
    plt.pause(1)



In [ ]:
"""
    3D state transition graph
"""

%matplotlib notebook
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import math
import collections 
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import clear_output
from DataManipulation import DataManipulation

plt.ion()
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111, projection='3d')

mpl.style.use('default')

# Reading output.csv and normalizing the data
data = DataManipulation("../server/output.csv")
NUM_OF_BINS = data.get_NUM_OF_BINS()
data.normalized_motor()
motor_values,sensor_values,us_values = data.normalized_0_to_1()

#Making a state transition between sensor and motor values
sensor_and_motor = data.convert_values_to_bins(sensor_values,motor_values)
transition_array = data.remove_continous_state()
transition_with_frequency = data.transition_frequency(transition_array)
most_visited_state = data.most_visited_state_transition(transition_with_frequency)

# Separating the values in the array to bins
xArray, yArray, zArray = data.digitize_total_values(most_visited_state, type='3d')

# Plotting the 3d quiver for every 3 values in the array with [A,B,C] 
# where A is source state, B is destination state, C is frequency
sc = ax.quiver(xArray[:-1:3], yArray[:-1:3], zArray[:-1:3], 
        xArray[1::3]-xArray[:-1:3],yArray[1::3]-yArray[:-1:3],zArray[1::3]-zArray[:-1:3],
        normalize=True,cmap=plt.cm.RdYlBu)

# Making a trend line in 3d graph
# taken from https://stackoverflow.com/questions/2298390/fitting-a-line-in-3d
data2 = np.concatenate((xArray.astype(float)[:, np.newaxis], 
                       yArray.astype(float)[:, np.newaxis], 
                       zArray.astype(float)[:, np.newaxis]), 
                      axis=1)

# Perturb with some Gaussian noise
data2 += np.random.normal(size=data2.shape) * 0.4

# Calculate the mean of the points, i.e. the 'center' of the cloud
datamean = data2.mean(axis=0)

# Do an SVD on the mean-centered data.
uu, dd, vv = np.linalg.svd(data2 - datamean)

# Now vv[0] contains the first principal component, i.e. the direction
# vector of the 'best fit' line in the least squares sense.

# Now generate some points along this best fit line, for plotting.

# I use -7, 7 since the spread of the data is roughly 14
# and we want it to have mean 0 (like the points we did
# the svd on). Also, it's a straight line, so we only need 2 points.
linepts = vv[0] * np.mgrid[-5:5:2j][:, np.newaxis]

# shift by the mean to get the line in the right place
linepts += datamean

ax.set_xlabel('Sensor')
ax.set_ylabel('Motor')
ax.set_zlabel('Ultrasound')
ax.set_xlim(0,10)
ax.set_ylim(0,10)
ax.set_zlim(0,10)
ax.set_title("3d normalized values between sensor, motor, and ultrasound")
ax.legend()

plt.show()


In [ ]:
"""
LIGHT-MOTOR MAP
#This graph represents the relationship between the combined amount of light detected
#and the combined speed of the motors with vectors. This was to show the transitions between
#different sensorimotor states in a 2D light-motor space.
"""
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import clear_output
import pandas as pd
import timeit
from DataManipulation import DataManipulation

#A timer to measure how long each iteration executes.
start_time = timeit.default_timer()

plt.ion()

while True:
    plt.figure(figsize=(20,20))
    #Read light and motor values, and add the left and right sides together for 
    #combined light and combined motor values
    listname = ["../server/angle0.csv", "../server/angle45.csv", "../server/angle-30.csv",
               "../server/angle-45.csv", "../server/angle-90.csv"]
    colorlist= ['red','blue','green','purple','orange']
    
    listplot = []
    xArray = []
    yArray = []
    for i in range(len(listname)):
        data = DataManipulation(listname[i])

        x = data.get_data()['left sensor'].values + data.get_data()['right sensor'].values
        y = data.get_data()['left motor'].values + data.get_data()['right motor'].values
        plt.subplot(211)
        #Add titles and labels
        plt.title("Motor Speed vs Relative Light w/ Vectors")
        plt.xlabel('Light')
        plt.ylabel('Motor Speed')
        plott, = plt.plot(x, y, label="Sensory-Motor path", alpha=0.5, color=colorlist[i])
        listplot.append(plott)
        xArray.extend(x)
        yArray.extend(y)
        #Add vectors to indicate direction of state transitions.
        plt.quiver(x[:-1], y[:-1], x[1:]-x[:-1], y[1:]-y[:-1], 
                   scale_units='xy', angles='xy', scale=1, alpha=0.5, color=colorlist[i])
    
    plt.legend(listplot, ['Angle 0', 'Angle 45', 'Angle -30', 'Angle -45', 'Angle -90'], prop={'size': 20})
    
    #Plotting an additional scatterplot for recording frequency of the visits to specific states
    plt.subplot(212)
    plt.hist2d(xArray, yArray, norm=mpl.colors.LogNorm(), bins=11, cmap=plt.cm.YlGnBu)
    plt.title("Motor Speed vs Relative Light w/ Colour Plot")
    plt.xlabel('Light')
    plt.ylabel('Motor Speed')   
    plt.colorbar(orientation='horizontal', label='Frequency of visits')
    
    #Pause for 1 second to give time for graph to update
    clear_output(wait=True)
    plt.pause(1)
    
    #Elapsed time printout for determining execution time of each iteration
    elapsed = timeit.default_timer() - start_time
    print(elapsed)
    start_time = timeit.default_timer()

while True:
    clear_output()
    plt.pause(1)

In [ ]:
from DataManipulation import DataManipulation
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import math
import collections
import pandas as pd
import networkx as nx
import mdp
from IPython.display import clear_output

plt.ion()

# Print the whole array rather than just a subset of the array
np.set_printoptions(threshold=np.nan,suppress=True)

# Necessary for livetracking
grid = plt.figure(figsize=(10,10))
#Add position for the plot with 211 being 2 row 1 column and position 1
fs = grid.add_subplot(511)
plt.title("Transition plot between Left Sensor and Right Sensor states")
plt.xlabel("Left Sensor")
plt.ylabel("Right Sensor")
ss = grid.add_subplot(512)
plt.title("Transition plot between Left Ultrasound and Right Ultrasound states")
plt.xlabel("Left Ultrasound")
plt.ylabel("Right Ultrasound")
ts = grid.add_subplot(513)
plt.title("Transition plot between Left Motor and Right Motor states")
plt.xlabel("Left Motor")
plt.ylabel("Right Motor")

# Reading output.csv and normalizing the data
data = DataManipulation("../server/output.csv")
NUM_OF_BINS = data.get_NUM_OF_BINS()
data.normalized_motor()
ls = np.floor(data.get_data()['left sensor'].values * NUM_OF_BINS).astype(int)
rs = np.floor(data.get_data()['right sensor'].values * NUM_OF_BINS).astype(int)
lus = np.floor(data.get_data()['left ultrasound sensor'].values * NUM_OF_BINS).astype(int)
rus = np.floor(data.get_data()['right ultrasound sensor'].values * NUM_OF_BINS).astype(int)
lm = np.floor((0.5*data.get_data()['left motor'].values + 0.5) * NUM_OF_BINS).astype(int)
rm = np.floor((0.5*data.get_data()['right motor'].values + 0.5) * NUM_OF_BINS).astype(int)

for i in range(len(ls)):
            if ls[i] == NUM_OF_BINS:
                ls[i] = NUM_OF_BINS-1
            if rs[i] == NUM_OF_BINS:
                rs[i] == NUM_OF_BINS-1

            if lus[i] == NUM_OF_BINS:
                lus[i] == NUM_OF_BINS-1
            if rus[i] == NUM_OF_BINS:
                rus[i] == NUM_OF_BINS-1
                
            if lm[i] == NUM_OF_BINS:
                lm[i] == NUM_OF_BINS-1
            if rm[i] == NUM_OF_BINS:
                rm[i] == NUM_OF_BINS-1
#print(lm,rm)

#Making a state transition between sensor and motor values
sensorimotor_state = data.convert_values_to_bins(ls,rs,lus,rus,lm,rm)
ls = data.remove_continous_state(ls)
rs = data.remove_continous_state(rs)
lus = data.remove_continous_state(lus)
rus = data.remove_continous_state(rus)
lm = data.remove_continous_state(lm)
rm = data.remove_continous_state(rm)

transition_array = data.remove_continous_state()
transition_with_frequency = data.transition_frequency(transition_array)

G = nx.DiGraph()
G.add_nodes_from(transition_with_frequency.keys())
for i,j in transition_with_frequency.items():
    for k,l in j.items():
        G.add_weighted_edges_from([(i, k, l)])
        
for i in nx.simple_cycles(G):
    if len(i) > 100:
        continue
    i.append(i[0])
    print(i)
    to_be_digitized = np.array(i)
    # ls, rs, lus , rus, lm, rm
    xArray2, yArray2, zArray2, wArray2, uArray2, vArray2 = data.digitize_cycle(to_be_digitized)
    
    for j in range(len(xArray2)-1):
            fs.annotate(
                        '',
                        xy=(xArray2[j+1],yArray2[j+1]),
                        xycoords='data',
                        xytext=(xArray2[j],yArray2[j]),
                        textcoords='data',
                        verticalalignment="center", 
                        horizontalalignment="center",
                        arrowprops=dict(facecolor='black', arrowstyle="-|>", 
                                        shrinkB=0.1, alpha=0.2)
                    )
            ts.annotate(
                        '',
                        xy=(uArray2[j+1],vArray2[j+1]),
                        xycoords='data',
                        xytext=(uArray2[j],vArray2[j]),
                        textcoords='data',
                        verticalalignment="center", 
                        horizontalalignment="center",
                        arrowprops=dict(facecolor='black', arrowstyle="-|>", 
                                        shrinkB=0.1, alpha=0.2)
            )
fs.set_xlim(-10,NUM_OF_BINS)
fs.set_ylim(-10,NUM_OF_BINS)
ss.set_xlim(-10,NUM_OF_BINS)
ss.set_ylim(-10,NUM_OF_BINS)
ts.set_xlim(-10,NUM_OF_BINS)
ts.set_ylim(-10,NUM_OF_BINS)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import math
import collections 
import networkx as nx
import mdp as mdp
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import clear_output
from DataManipulation import DataManipulation

# Reading output.csv and normalizing the data
data = DataManipulation("../server/output2.csv")
NUM_OF_BINS = data.get_NUM_OF_BINS()
data.normalized_motor()
ls = np.floor(data.get_data()['left sensor'].values * NUM_OF_BINS).astype(int)
rs = np.floor(data.get_data()['right sensor'].values * NUM_OF_BINS).astype(int)
lus = np.floor(data.get_data()['left ultrasound sensor'].values * NUM_OF_BINS).astype(int)
rus = np.floor(data.get_data()['right ultrasound sensor'].values * NUM_OF_BINS).astype(int)
lm = np.floor((0.5*data.get_data()['left motor'].values + 0.5) * NUM_OF_BINS).astype(int)
rm = np.floor((0.5*data.get_data()['right motor'].values + 0.5) * NUM_OF_BINS).astype(int)

#plt.autoscale(enable=True)
for i in range(len(ls)):
            if ls[i] == NUM_OF_BINS:
                ls[i] = NUM_OF_BINS-1
            if rs[i] == NUM_OF_BINS:
                rs[i] == NUM_OF_BINS-1

            if lus[i] == NUM_OF_BINS:
                lus[i] == NUM_OF_BINS-1
            if rus[i] == NUM_OF_BINS:
                rus[i] == NUM_OF_BINS-1
                
            if lm[i] == NUM_OF_BINS:
                lm[i] == NUM_OF_BINS-1
            if rm[i] == NUM_OF_BINS:
                rm[i] == NUM_OF_BINS-1
#print(lm,rm)

#Making a state transition between sensor and motor values
sensorimotor_state = data.convert_values_to_bins(ls,rs,lus,rus,lm,rm)
ls = data.remove_continous_state(ls)
rs = data.remove_continous_state(rs)
lus = data.remove_continous_state(lus)
rus = data.remove_continous_state(rus)
lm = data.remove_continous_state(lm)
rm = data.remove_continous_state(rm)

transition_array = data.remove_continous_state()
transition_with_frequency = data.transition_frequency(transition_array)

G = nx.DiGraph()
G.add_nodes_from(transition_with_frequency.keys())
for i,j in transition_with_frequency.items():
    for k,l in j.items():
        G.add_weighted_edges_from([(i, k, l)])

for i in nx.simple_cycles(G):
    if len(i) > 50:
        continue
    i.append(i[0])
    to_be_digitized = np.array(i)
    # ls, rs, lus , rus, lm, rm
    xArray2, yArray2, zArray2, wArray2, uArray2, vArray2 = data.digitize_cycle(to_be_digitized)
    
    dimensions = np.array(list(zip(xArray2,yArray2,zArray2,wArray2,uArray2,vArray2)))
    pca_nodes = mdp.nodes.PCANode(dtype='float64',svd=True)
    pca_result = pca_nodes.execute(dimensions)
    projection_matrix = pca_nodes.get_projmatrix()
    
    all_zeros = not pca_result[:,1].any()
    if all_zeros:
        continue
    print(len(i))
    print(dimensions)
    
    print(pca_result)
    print()
    print(projection_matrix)
    
    for j in range (len(projection_matrix)-1):
        plt.annotate(
                            '',
                            xy=(projection_matrix[:,0][j+1],projection_matrix[:,1][j+1]),
                            xycoords='data',
                            xytext=(projection_matrix[:,0][j],projection_matrix[:,1][j]),
                            textcoords='data',
                            verticalalignment="center", 
                            horizontalalignment="center",
                            arrowprops=dict(facecolor='black', arrowstyle="-|>", 
                                            shrinkB=0.1, alpha=0.2)
                        )
    plt.xlim(-NUM_OF_BINS,NUM_OF_BINS)
    plt.ylim(-NUM_OF_BINS,NUM_OF_BINS)
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.title('PC1 VS PC2')
    plt.show()
    print()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import math
import collections 
import networkx as nx
import mdp as mdp
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import clear_output
from DataManipulation import DataManipulation

# Reading output.csv and normalizing the data
data = DataManipulation("../server/output2.csv")
NUM_OF_BINS = data.get_NUM_OF_BINS()
data.normalized_motor()
ls = np.floor(data.get_data()['left sensor'].values * NUM_OF_BINS).astype(int)
rs = np.floor(data.get_data()['right sensor'].values * NUM_OF_BINS).astype(int)
lus = np.floor(data.get_data()['left ultrasound sensor'].values * NUM_OF_BINS).astype(int)
rus = np.floor(data.get_data()['right ultrasound sensor'].values * NUM_OF_BINS).astype(int)
lm = np.floor((0.5*data.get_data()['left motor'].values + 0.5) * NUM_OF_BINS).astype(int)
rm = np.floor((0.5*data.get_data()['right motor'].values + 0.5) * NUM_OF_BINS).astype(int)

#plt.autoscale(enable=True)
for i in range(len(ls)):
            if ls[i] == NUM_OF_BINS:
                ls[i] = NUM_OF_BINS-1
            if rs[i] == NUM_OF_BINS:
                rs[i] == NUM_OF_BINS-1

            if lus[i] == NUM_OF_BINS:
                lus[i] == NUM_OF_BINS-1
            if rus[i] == NUM_OF_BINS:
                rus[i] == NUM_OF_BINS-1
                
            if lm[i] == NUM_OF_BINS:
                lm[i] == NUM_OF_BINS-1
            if rm[i] == NUM_OF_BINS:
                rm[i] == NUM_OF_BINS-1
#print(lm,rm)

#Making a state transition between sensor and motor values
sensorimotor_state = data.convert_values_to_bins(ls,rs,lus,rus,lm,rm)
ls = data.remove_continous_state(ls)
rs = data.remove_continous_state(rs)
lus = data.remove_continous_state(lus)
rus = data.remove_continous_state(rus)
lm = data.remove_continous_state(lm)
rm = data.remove_continous_state(rm)

transition_array = data.remove_continous_state()
transition_with_frequency = data.transition_frequency(transition_array)

G = nx.DiGraph()
G.add_nodes_from(transition_with_frequency.keys())
for i,j in transition_with_frequency.items():
    for k,l in j.items():
        G.add_weighted_edges_from([(i, k, l)])

for i in nx.simple_cycles(G):
    if len(i) > 50:
        continue
    i.append(i[0])
    to_be_digitized = np.array(i)
    # ls, rs, lus , rus, lm, rm
    xArray2, yArray2, zArray2, wArray2, uArray2, vArray2 = data.digitize_cycle(to_be_digitized)
    
    dimensions = np.array(list(zip(xArray2,yArray2,zArray2,wArray2,uArray2,vArray2)))
    pca_nodes = mdp.nodes.PCANode(dtype='float64',svd=True)
    pca_result = pca_nodes.execute(dimensions)
    projection_matrix = pca_nodes.get_projmatrix()
    
    all_zeros = not pca_result[:,2].any()
    if all_zeros:
        continue

    print(len(i))
    print(dimensions)
    
    print(pca_result)
    print()
    print(projection_matrix)

    for j in range (len(pca_result)-1):
        plt.annotate(
                            '',
                            xy=(pca_result[:,0][j+1],pca_result[:,2][j+1]),
                            xycoords='data',
                            xytext=(pca_result[:,0][j],pca_result[:,2][j]),
                            textcoords='data',
                            verticalalignment="center", 
                            horizontalalignment="center",
                            arrowprops=dict(facecolor='black', arrowstyle="-|>", 
                                            shrinkB=0.1, alpha=0.2)
                        )
    plt.xlim(-NUM_OF_BINS,NUM_OF_BINS)
    plt.ylim(-NUM_OF_BINS,NUM_OF_BINS)
    plt.xlabel('PC1')
    plt.ylabel('PC3')
    plt.title('PC1 VS PC3')
    plt.show()
    print()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import math
import collections
import networkx as nx
import mdp as mdp
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import clear_output
from DataManipulation import DataManipulation

# Reading output.csv and normalizing the data
data = DataManipulation("../server/output2.csv")
NUM_OF_BINS = data.get_NUM_OF_BINS()
data.normalized_motor()
ls = np.floor(data.get_data()['left sensor'].values * NUM_OF_BINS).astype(int)
rs = np.floor(data.get_data()['right sensor'].values * NUM_OF_BINS).astype(int)
lus = np.floor(data.get_data()['left ultrasound sensor'].values * NUM_OF_BINS).astype(int)
rus = np.floor(data.get_data()['right ultrasound sensor'].values * NUM_OF_BINS).astype(int)
lm = np.floor((0.5*data.get_data()['left motor'].values + 0.5) * NUM_OF_BINS).astype(int)
rm = np.floor((0.5*data.get_data()['right motor'].values + 0.5) * NUM_OF_BINS).astype(int)

#plt.autoscale(enable=True)
for i in range(len(ls)):
            if ls[i] == NUM_OF_BINS:
                ls[i] = NUM_OF_BINS-1
            if rs[i] == NUM_OF_BINS:
                rs[i] == NUM_OF_BINS-1

            if lus[i] == NUM_OF_BINS:
                lus[i] == NUM_OF_BINS-1
            if rus[i] == NUM_OF_BINS:
                rus[i] == NUM_OF_BINS-1
                
            if lm[i] == NUM_OF_BINS:
                lm[i] == NUM_OF_BINS-1
            if rm[i] == NUM_OF_BINS:
                rm[i] == NUM_OF_BINS-1
#print(lm,rm)

#Making a state transition between sensor and motor values
sensorimotor_state = data.convert_values_to_bins(ls,rs,lus,rus,lm,rm)
ls = data.remove_continous_state(ls)
rs = data.remove_continous_state(rs)
lus = data.remove_continous_state(lus)
rus = data.remove_continous_state(rus)
lm = data.remove_continous_state(lm)
rm = data.remove_continous_state(rm)

transition_array = data.remove_continous_state()
transition_with_frequency = data.transition_frequency(transition_array)

G = nx.DiGraph()
G.add_nodes_from(transition_with_frequency.keys())
for i,j in transition_with_frequency.items():
    for k,l in j.items():
        G.add_weighted_edges_from([(i, k, l)])

for i in nx.simple_cycles(G):
    if len(i) > 50:
        continue
    i.append(i[0])
    to_be_digitized = np.array(i)
    # ls, rs, lus , rus, lm, rm
    xArray2, yArray2, zArray2, wArray2, uArray2, vArray2 = data.digitize_cycle(to_be_digitized)
        
    dimensions = np.array(list(zip(xArray2,yArray2,zArray2,wArray2,uArray2,vArray2)))
    pca_nodes = mdp.nodes.PCANode(dtype='float64',svd=True)
    pca_result = pca_nodes.execute(dimensions)
    projection_matrix = pca_nodes.get_projmatrix()
    
    all_zeros = not pca_result[:,2].any()
    if all_zeros:
        continue

    print(len(i))
    print(dimensions)
    
    print(pca_result)
    print()
    print(projection_matrix)
    
    for j in range (len(pca_result)-1):
        plt.annotate(
                            '',
                            xy=(pca_result[:,1][j+1],pca_result[:,2][j+1]),
                            xycoords='data',
                            xytext=(pca_result[:,1][j],pca_result[:,2][j]),
                            textcoords='data',
                            verticalalignment="center", 
                            horizontalalignment="center",
                            arrowprops=dict(facecolor='black', arrowstyle="-|>", 
                                            shrinkB=0.1, alpha=0.2)
                        )
    plt.xlim(-10,10)
    plt.ylim(-10,10)
    plt.xlabel('PC2')
    plt.ylabel('PC3')
    plt.title('PC2 VS PC3')
    plt.show()
    print()

In [ ]:
%matplotlib notebook
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import math
import collections 
import networkx as nx
import mdp as mdp
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import clear_output
from DataManipulation import DataManipulation

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111, projection='3d')

# Reading output.csv and normalizing the data
data = DataManipulation("../server/output2.csv")
NUM_OF_BINS = data.get_NUM_OF_BINS()
data.normalized_motor()
ls = np.floor(data.get_data()['left sensor'].values * NUM_OF_BINS).astype(int)
rs = np.floor(data.get_data()['right sensor'].values * NUM_OF_BINS).astype(int)
lus = np.floor(data.get_data()['left ultrasound sensor'].values * NUM_OF_BINS).astype(int)
rus = np.floor(data.get_data()['right ultrasound sensor'].values * NUM_OF_BINS).astype(int)
lm = np.floor((0.5*data.get_data()['left motor'].values + 0.5) * NUM_OF_BINS).astype(int)
rm = np.floor((0.5*data.get_data()['right motor'].values + 0.5) * NUM_OF_BINS).astype(int)

#plt.autoscale(enable=True)
for i in range(len(ls)):
            if ls[i] == NUM_OF_BINS:
                ls[i] = NUM_OF_BINS-1
            if rs[i] == NUM_OF_BINS:
                rs[i] == NUM_OF_BINS-1

            if lus[i] == NUM_OF_BINS:
                lus[i] == NUM_OF_BINS-1
            if rus[i] == NUM_OF_BINS:
                rus[i] == NUM_OF_BINS-1
                
            if lm[i] == NUM_OF_BINS:
                lm[i] == NUM_OF_BINS-1
            if rm[i] == NUM_OF_BINS:
                rm[i] == NUM_OF_BINS-1
#print(lm,rm)

#Making a state transition between sensor and motor values
sensorimotor_state = data.convert_values_to_bins(ls,rs,lus,rus,lm,rm)
ls = data.remove_continous_state(ls)
rs = data.remove_continous_state(rs)
lus = data.remove_continous_state(lus)
rus = data.remove_continous_state(rus)
lm = data.remove_continous_state(lm)
rm = data.remove_continous_state(rm)

transition_array = data.remove_continous_state()
transition_with_frequency = data.transition_frequency(transition_array)

G = nx.DiGraph()
G.add_nodes_from(transition_with_frequency.keys())
for i,j in transition_with_frequency.items():
    for k,l in j.items():
        G.add_weighted_edges_from([(i, k, l)])

for i in nx.simple_cycles(G):
    if len(i) > 50:
        continue
    i.append(i[0])
    to_be_digitized = np.array(i)
    # ls, rs, lus , rus, lm, rm
    xArray2, yArray2, zArray2, wArray2, uArray2, vArray2 = data.digitize_cycle(to_be_digitized)
        
    dimensions = np.array(list(zip(xArray2,yArray2,zArray2,wArray2,uArray2,vArray2)))
    pca_nodes = mdp.nodes.PCANode(dtype='float64',svd=True)
    pca_result = pca_nodes.execute(dimensions)
    projection_matrix = pca_nodes.get_projmatrix()

    print(len(i))
    print(dimensions)
    
    print(pca_result)
    print()
    print(projection_matrix)
    
    for j in range (len(pca_result)-1):
        ax.quiver(pca_result[:,0][j],
                 pca_result[:,1][j],
                 pca_result[:,2][j],
                 pca_result[:,0][j+1],
                 pca_result[:,1][j+1],
                 pca_result[:,2][j+1],
                 normalize=True)

    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    ax.set_zlabel('PC3')
    fig.suptitle('PCA between left light, right light and total motor')
    plt.show()
    print()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import math
import collections 
import networkx as nx
import mdp as mdp
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import clear_output
from DataManipulation import DataManipulation

# Reading output.csv and normalizing the data
data = DataManipulation("../server/output2.csv")
NUM_OF_BINS = data.get_NUM_OF_BINS()
data.normalized_motor()
ls = np.floor(data.get_data()['left sensor'].values * NUM_OF_BINS).astype(int)
rs = np.floor(data.get_data()['right sensor'].values * NUM_OF_BINS).astype(int)
lus = np.floor(data.get_data()['left ultrasound sensor'].values * NUM_OF_BINS).astype(int)
rus = np.floor(data.get_data()['right ultrasound sensor'].values * NUM_OF_BINS).astype(int)
lm = np.floor((0.5*data.get_data()['left motor'].values + 0.5) * NUM_OF_BINS).astype(int)
rm = np.floor((0.5*data.get_data()['right motor'].values + 0.5) * NUM_OF_BINS).astype(int)

#plt.autoscale(enable=True)
for i in range(len(ls)):
            if ls[i] == NUM_OF_BINS:
                ls[i] = NUM_OF_BINS-1
            if rs[i] == NUM_OF_BINS:
                rs[i] == NUM_OF_BINS-1

            if lus[i] == NUM_OF_BINS:
                lus[i] == NUM_OF_BINS-1
            if rus[i] == NUM_OF_BINS:
                rus[i] == NUM_OF_BINS-1
                
            if lm[i] == NUM_OF_BINS:
                lm[i] == NUM_OF_BINS-1
            if rm[i] == NUM_OF_BINS:
                rm[i] == NUM_OF_BINS-1
#print(lm,rm)

#Making a state transition between sensor and motor values
sensorimotor_state = data.convert_values_to_bins(ls,rs,lus,rus,lm,rm)
ls = data.remove_continous_state(ls)
rs = data.remove_continous_state(rs)
lus = data.remove_continous_state(lus)
rus = data.remove_continous_state(rus)
lm = data.remove_continous_state(lm)
rm = data.remove_continous_state(rm)

transition_array = data.remove_continous_state()
transition_with_frequency = data.transition_frequency(transition_array)

G = nx.DiGraph()
G.add_nodes_from(transition_with_frequency.keys())
for i,j in transition_with_frequency.items():
    for k,l in j.items():
        G.add_weighted_edges_from([(i, k, l)])

for i in nx.simple_cycles(G):
    if len(i) > 50:
        continue
    i.append(i[0])
    to_be_digitized = np.array(i)
    # ls, rs, lus , rus, lm, rm
    xArray2, yArray2, zArray2, wArray2, uArray2, vArray2 = data.digitize_cycle(to_be_digitized)
        
    dimensions = np.array(list(zip(xArray2,yArray2,zArray2,wArray2,uArray2,vArray2)))
    pca_nodes = mdp.nodes.PCANode(dtype='float64',svd=True)
    pca_result = pca_nodes.execute(dimensions)
    projection_matrix = pca_nodes.get_projmatrix()

    print(len(i))
    print(dimensions)
    
    print(pca_result)
    print()
    print(projection_matrix)
    
    for j in range (len(pca_result)-1):
        plt.scatter(pca_result[:,0][j], [1]*pca_result[:,0][j].size)
        plt.scatter(pca_result[:,1][j], [2]*pca_result[:,1][j].size)
        plt.scatter(pca_result[:,2][j], [3]*pca_result[:,2][j].size)
        plt.scatter(pca_result[:,3][j], [4]*pca_result[:,3][j].size)
        plt.scatter(pca_result[:,4][j], [5]*pca_result[:,4][j].size)
        plt.scatter(pca_result[:,5][j], [6]*pca_result[:,5][j].size)

    plt.xlim(-10,10)
    plt.ylim(0,7)
    plt.xlabel('PCA line')
    plt.ylabel('PC number')
    plt.title('Full PCA line')
    plt.show()
    print()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import math
import collections 
import networkx as nx
import mdp as mdp
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import clear_output
from DataManipulation import DataManipulation

# Reading output.csv and normalizing the data
data = DataManipulation("../server/output2.csv")
NUM_OF_BINS = data.get_NUM_OF_BINS()
data.normalized_motor()
ls = np.floor(data.get_data()['left sensor'].values * NUM_OF_BINS).astype(int)
rs = np.floor(data.get_data()['right sensor'].values * NUM_OF_BINS).astype(int)
lus = np.floor(data.get_data()['left ultrasound sensor'].values * NUM_OF_BINS).astype(int)
rus = np.floor(data.get_data()['right ultrasound sensor'].values * NUM_OF_BINS).astype(int)
lm = np.floor((0.5*data.get_data()['left motor'].values + 0.5) * NUM_OF_BINS).astype(int)
rm = np.floor((0.5*data.get_data()['right motor'].values + 0.5) * NUM_OF_BINS).astype(int)

#plt.autoscale(enable=True)
for i in range(len(ls)):
            if ls[i] == NUM_OF_BINS:
                ls[i] = NUM_OF_BINS-1
            if rs[i] == NUM_OF_BINS:
                rs[i] == NUM_OF_BINS-1

            if lus[i] == NUM_OF_BINS:
                lus[i] == NUM_OF_BINS-1
            if rus[i] == NUM_OF_BINS:
                rus[i] == NUM_OF_BINS-1
                
            if lm[i] == NUM_OF_BINS:
                lm[i] == NUM_OF_BINS-1
            if rm[i] == NUM_OF_BINS:
                rm[i] == NUM_OF_BINS-1
#print(lm,rm)

#Making a state transition between sensor and motor values
sensorimotor_state = data.convert_values_to_bins(ls,rs,lus,rus,lm,rm)
ls = data.remove_continous_state(ls)
rs = data.remove_continous_state(rs)
lus = data.remove_continous_state(lus)
rus = data.remove_continous_state(rus)
lm = data.remove_continous_state(lm)
rm = data.remove_continous_state(rm)
transition_array = data.remove_continous_state()
transition_with_frequency = data.transition_frequency(transition_array)
G = nx.DiGraph()
G.add_nodes_from(transition_with_frequency.keys())
for i,j in transition_with_frequency.items():
    for k,l in j.items():
        G.add_weighted_edges_from([(i, k, l)])

for i in nx.simple_cycles(G):
    if len(i) > 50:
        continue
    i.append(i[0])
    to_be_digitized = np.array(i)
    # ls, rs, lus , rus, lm, rm
    xArray2, yArray2, zArray2, wArray2, uArray2, vArray2 = data.digitize_cycle(to_be_digitized)
        
    dimensions = np.array(list(zip(xArray2,yArray2,zArray2,wArray2,uArray2,vArray2)))
    pca_nodes = mdp.nodes.PCANode(dtype='float64',svd=True)
    pca_result = pca_nodes.execute(dimensions)
    projection_matrix = pca_nodes.get_projmatrix()

    print(len(i))
    print(dimensions)
    
    print(pca_result)
    print()
    print(projection_matrix)
    
    for j in range (len(pca_result)-1):
        plt.scatter(pca_result[:,0][j], [1]*pca_result[:,0][j].size)
        plt.scatter(pca_result[:,1][j], [2]*pca_result[:,1][j].size)
        plt.scatter(pca_result[:,2][j], [3]*pca_result[:,2][j].size)
        plt.scatter(pca_result[:,3][j], [4]*pca_result[:,3][j].size)
        plt.scatter(pca_result[:,4][j], [5]*pca_result[:,4][j].size)
        plt.scatter(pca_result[:,5][j], [6]*pca_result[:,5][j].size)

    plt.xlim(-10,10)
    plt.ylim(0,7)
    plt.xlabel('PCA line')
    plt.ylabel('PC number')
    plt.title('Full PCA line')
    plt.show()
    print()